In [1]:
import os
import sys
import json
import time
import math
import gspread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from scipy import integrate, signal, stats, fftpack
from glob import glob
from datetime import date
import mne

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from tbd_eeg.tbd_eeg.data_analysis.Utilities.utilities import (
    get_stim_events,
    get_evoked_traces,
    get_evoked_firing_rates,
    find_nearest_ind
)
from PCIst.PCIst.pci_st import calc_PCIst, dimensionality_reduction
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [4]:
%matplotlib notebook

Load CCF for identifying cortical areas

In [5]:
mcc = MouseConnectivityCache(resolution=10)
str_tree = mcc.get_structure_tree()

#### Functions

### Subjects

In [6]:
## All subjects ##
group_title = 'all'
PCImeancolor = 'darkblue'

sub_dirs = {
    '551399': r'F:\EEG_exp\mouse551399\estim_2021-01-28_13-59-09\experiment1\recording1',
    '551397': r'F:\EEG_exp\mouse551397\estim_vis_2021-02-11_10-45-23\experiment1\recording1',
    '569062': r'F:\EEG_exp\mouse569062\estim_vis_2021-02-18_11-17-51\experiment1\recording1',
    '569068': r'F:\EEG_exp\mouse569068\estim_vis_2021-03-04_10-51-38\experiment1\recording1',
    '569069': r'F:\EEG_exp\mouse569069\estim_vis2_2021-03-12_10-52-44\experiment1\recording1',
    '569064': r'F:\EEG_exp\mouse569064\estim_vis_2021-04-08_10-28-24\experiment1\recording1',
    '569073a': r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1',
    '571619': r'F:\EEG_exp\mouse571619\estim2_2021-03-19_10-09-01\experiment1\recording1',
#     '569070': r'F:\EEG_exp\mouse569070\estim1_2021-04-01_10-27-33\experiment1\recording1', # no area assignments
#     '521885': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521885\estim1_2020-07-09_14-23-49\experiment1\recording1',
#     '521886': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521886\estim1_2020-07-16_13-37-02\experiment1\recording1',
#     '521887': r'C:\Users\lesliec\OneDrive - Allen Institute\data\mouse521887\estim1_2020-07-30_11-25-05\experiment1\recordingfull',
#     '543393': r'F:\EEG_exp\mouse543393\estim1_2020-08-20_13-24-53\experiment1\recording1', # no area assignments
#     '543394': r'F:\EEG_exp\mouse543394\estim1_2020-08-27_14-32-00\experiment1\recording1', # no area assignments
    '543396': r'F:\EEG_exp\mouse543396\estim_vis1_2020-09-18_12-04-46\experiment1\recording1', # too few resting trials
    '546655': r'F:\EEG_exp\mouse546655\estim_vis_2020-10-23_11-01-14\experiment1\recording1',
    '571618': r'F:\EEG_exp\mouse571618\estim1_2021-04-29_12-28-54\experiment1\recording1', # too many bad chs
    '575102': r'F:\EEG_exp\mouse575102\estim_vis_2021-06-03_11-25-01\experiment1\recording1',
    '575100': r'F:\EEG_exp\mouse575100\estim_vis_2021-06-10_11-39-06\experiment1\recording1',
    '569073b': r'F:\EEG_exp\mouse569073\estim_2021-04-16_10-42-44\experiment1\recording1',
    '569072': r'F:\EEG_exp\mouse569072\estim_vis_2021-04-22_10-26-58\experiment1\recording1',
    '571620': r'F:\EEG_exp\mouse571620\estim_vis_2021-05-13_11-33-47\experiment1\recording1',
    '582387': r'F:\EEG_exp\mouse582387\estim_vis_2021-07-22_09-59-47\experiment1\recording1', # too many bad chs
    '586466': r'F:\EEG_exp\mouse586466\estim_vis_2021-07-29_10-37-41\experiment1\recording1', # no area assignments YET
    '586468': r'F:\EEG_exp\mouse586468\estim_vis_2021-08-19_11-36-53\experiment1\recording1', # no area assignments YET (tissuecyte)
#     '590479': r'', # no area assignments YET (tissuecyte)
}
## 9/2/21: all subjects except for 586466 and 586468 (who don't have area assignments yet anyway) ##

### Load experiments

In [7]:
sub_exp = {}
for mousei, recdiri in sub_dirs.items():
    sub_exp[mousei] = EEGexp(recdiri, preprocess=False, make_stim_csv=False)

Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical and sensory stimulation
Body camera file not found.
Pupil camera file not found.
Experiment type: electrical and sensory stimulation
Experiment type: electrical and sensory stimulation
Experiment type: electrical and sensory stimulation
Experiment type: electrical and sensory stimulation
SomnoSuite log file not found.
Experiment type: electrical and sensory stimulation
Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical stimulation
Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical stimulation
SomnoSuite log file not found.
Experiment type: electrical and sensory stimulation
SomnoSuite 

### Load probe data and gather a list of unit areas

In [10]:
mega_areas_list = []
for mousei, expi in sub_exp.items():
    print(mousei)
    
    ### Load unit data ###
    probe_list = [x.replace('_sorted', '') for x in expi.experiment_data if 'probe' in x]
    sub_areas_list = []
    for probei in probe_list:
        print(' {}'.format(probei))

        ## Load probe_info.json ##
        with open(expi.ephys_params[probei]['probe_info']) as data_file:
            data = json.load(data_file)
        npx_allch = np.array(data['channel']) # this is an array from 0 to 384
        surface_ch = int(data['surface_channel']) # the electrode we said was at the brain surface
        allch_z = np.array(data['vertical_pos']) # vertical position of each electrode (um), relative to the tip (ch 0 is 20 um from tip)
        ref_mask = np.array(data['mask']) # contains a False for Npx reference channels

        ## Get all ch depths ##
        npx_chs = np.array([x for x in npx_allch if ref_mask[x] and x <= surface_ch]) 
        all_ch_depths = allch_z[surface_ch] - allch_z

        ## Select units and get peak chs ##
        select_units, peak_chs, unit_metrics = expi.get_probe_units(probei)

        ## Sort units ##
        all_units = select_units[np.squeeze(np.argsort(peak_chs))]
        all_units_chs = peak_chs[np.squeeze(np.argsort(peak_chs))]
        if 'area' in unit_metrics.columns:
            all_units_areas = unit_metrics.area.values[np.squeeze(np.argsort(peak_chs))]
            sub_areas_list.append(np.unique(all_units_areas))
        else:
            print('  unit metrics file does not have area assignments.')

    if len(sub_areas_list) > 0:
        mega_areas_list.append(np.unique(np.concatenate(sub_areas_list)))
    
mega_areas_list = np.unique(np.concatenate(mega_areas_list))

551399
 probeB
 probeF
551397
 probeB
 probeF
569062
 probeB
 probeD
 probeF
569068
 probeB
 probeD
 probeF
569069
 probeB
 probeC
 probeF
569064
 probeB
 probeC
 probeF
569073a
 probeB
 probeC
 probeF
571619
 probeB
 probeC
 probeF
569070
 probeB
  unit metrics file does not have area assignments.
 probeC
  unit metrics file does not have area assignments.
 probeF
  unit metrics file does not have area assignments.
521885
521886
521887
543393
 probeB
  unit metrics file does not have area assignments.
543394
 probeB
  unit metrics file does not have area assignments.
543396
 probeB
 probeC
546655
 probeB
 probeC
571618
 probeB
 probeC
 probeF
575102
 probeB
 probeC
 probeF
575100
 probeB
 probeC
  unit metrics file does not have area assignments.
 probeF
569073b
 probeB
 probeC
 probeF
569072
 probeB
 probeC
 probeF
571620
 probeB
 probeC
 probeF
582387
 probeB
  unit metrics file does not have area assignments.
 probeC
  unit metrics file does not have area assignments.
 probeF
  uni

## List all brain regions we have recorded units from in all experiments

In [11]:
print(mega_areas_list)

['ACAd5' 'ACAd6a' 'ACAd6b' 'ACAv1' 'ACAv2/3' 'ACAv5' 'ACAv6a' 'ACAv6b'
 'APN' 'AV' 'CA1' 'CA2' 'CA3' 'CL' 'CP' 'DG-mo' 'DG-po' 'DG-sg' 'Eth'
 'HPF' 'IG' 'IGL' 'ILA1' 'LD' 'LGd-co' 'LGd-ip' 'LGd-sh' 'LGv' 'LP' 'MB'
 'MD' 'MGd' 'MGm' 'MGv' 'MOp2/3' 'MOp5' 'MOp6a' 'MOp6b' 'MOs2/3' 'MOs5'
 'MOs6a' 'MOs6b' 'MRN' 'PCN' 'PIL' 'PL1' 'PL2/3' 'PL5' 'PL6a' 'PO' 'PP'
 'ProS' 'RSPagl5' 'RSPagl6a' 'RSPd5' 'RSPd6a' 'RSPd6b' 'RT' 'SCig' 'SGN'
 'SNr' 'SSp-bfd1' 'SSp-bfd2/3' 'SSp-bfd4' 'SSp-bfd5' 'SSp-bfd6a'
 'SSp-bfd6b' 'SSp-ll4' 'SSp-ll5' 'SSp-ll6a' 'SSp-ll6b' 'SSp-tr2/3'
 'SSp-tr4' 'SSp-tr5' 'SSp-tr6a' 'SUB' 'TH' 'VAL' 'VISam5' 'VISam6a'
 'VISam6b' 'VISp1' 'VISp2/3' 'VISp4' 'VISp5' 'VISp6a' 'VISp6b' 'VISpm2/3'
 'VISpm5' 'VISpm6a' 'VISpm6b' 'VISrl2/3' 'VISrl4' 'VISrl5' 'VISrl6a'
 'VISrl6b' 'VPL' 'VPM' 'ZI' 'alv' 'ar' 'bsc' 'ccb' 'ccg' 'cing' 'dhc' 'em'
 'fa' 'fi' 'fiber tracts' 'fp' 'int' 'or' 'root' 'scwm']


In [24]:
str_tree.get_structures_by_acronym(['root'])

[{'acronym': 'root',
  'graph_id': 1,
  'graph_order': 0,
  'id': 997,
  'name': 'root',
  'structure_id_path': [997],
  'structure_set_ids': [691663206],
  'rgb_triplet': [255, 255, 255]}]

In [15]:
str_tree.get_structures_by_id([695])

[{'acronym': 'CTXpl',
  'graph_id': 1,
  'graph_order': 4,
  'id': 695,
  'name': 'Cortical plate',
  'structure_id_path': [997, 8, 567, 688, 695],
  'structure_set_ids': [112905828,
   691663206,
   12,
   184527634,
   112905813,
   114512891,
   114512892],
  'rgb_triplet': [112, 255, 112]}]